In [14]:
import torch
import torch.nn.functional as F

# t = torch.Tensor([[1.9392, -1.9266, 0.9664], 
#             [0.0000, -1.9266, 0.9664], 
#             [0.0000, -0.0000, 0.9664]]) 

def covariance_formula(ret):
    idx = torch.tensor([[0, 1, 2, 1, 2, 2], [0, 1, 2, 0, 0, 1]], dtype=torch.int64)
    A = torch.zeros((3, 3))
    # ret = torch.rand((6,))# if False else torch.tensor([0, 0, 0, -torch.inf, -torch.inf, -torch.inf], dtype=torch.float32)
    # print(ret.dtype)
    A[idx[0], idx[1]] = ret.exp()
    S = A @ A.T

    loss = 0.5 * ((2 * ret).exp().sum() - 2 * ret[:3].sum() - 3)
    test_loss = 0.5 * (torch.trace(S) - torch.log(torch.linalg.det(S)) - 3)
    assert torch.isclose(loss, test_loss)
    return loss

def naive_batch_covariance_formula(ret):
    losses = []
    for i in range(len(ret)):
        loss = covariance_formula(ret[i])
        losses.append(loss)
    return torch.stack(losses, dim=0)

def test_covariance_formula(input):
    idx = torch.tensor([[0, 1, 2], [0, 1, 2]], dtype=torch.int64)

    # TODO: add requires grad.
    A = torch.zeros((input.shape[0], 3, 3))
    # ret = torch.rand((6,))# if False else torch.tensor([0, 0, 0, -torch.inf, -torch.inf, -torch.inf], dtype=torch.float32)
    # print(ret.dtype)
    A[:, idx[0], idx[1]] = input.exp().sqrt()
    S = A @ A.transpose(1, 2).contiguous()

    loss = 0.5 * (input.exp().sum(dim=1) - input[:, :3].sum(dim=1) - 3)
    test_loss = 0.5 * (S.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1) - torch.log(torch.linalg.det(S)) - 3)
    real_loss = 0.5 * torch.sum(input.exp() - 1 - input, dim = 1)
    print(f'loss={loss}\ntest_loss={test_loss}\nreal_loss={real_loss}')
    # assert torch.allclose(loss, test_loss) and torch.allclose(loss, real_loss)
    return loss

def batch_covariance_formula(input):
    idx = torch.tensor([[0, 1, 2, 1, 2, 2], [0, 1, 2, 0, 0, 1]], dtype=torch.int64)

    # TODO: add requires grad.
    A = torch.zeros((input.shape[0], 3, 3))
    # ret = torch.rand((6,))# if False else torch.tensor([0, 0, 0, -torch.inf, -torch.inf, -torch.inf], dtype=torch.float32)
    # print(ret.dtype)
    print(f'input={input}')
    A[:, idx[0], idx[1]] = (0.5 * input).exp()
    S = A @ A.transpose(1, 2).contiguous()

    loss = 0.5 * (input.exp().sum(dim=1) - input[:, :3].sum(dim=1) - 3)
    test_loss = torch.mean(0.5 * (S.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1) - torch.log(torch.linalg.det(S)) - 3))
    print(f'loss={loss}, test_loss={test_loss}')
    assert torch.allclose(loss, test_loss)
    return loss

batch_size = 8
for i in range(100):
    input = torch.rand((batch_size, 6))
    # naive_output = naive_batch_covariance_formula(input)
    # output = batch_covariance_formula(input)
    loss = batch_covariance_formula(input)
    assert torch.max(loss) < 1.0
    # print(loss)
    # assert loss < 1.0
# assert torch.allclose(output, naive_output)
# naive_output, output

input=tensor([[0.6780, 0.5653, 0.9281],
        [0.4807, 0.1337, 0.8328],
        [0.0082, 0.6732, 0.0724],
        [0.7547, 0.5314, 0.3209],
        [0.1066, 0.7883, 0.8796],
        [0.8593, 0.1064, 0.0486],
        [0.5722, 0.0228, 0.9265],
        [0.8189, 0.9836, 0.1628]])


RuntimeError: shape mismatch: value tensor of shape [8, 3] cannot be broadcast to indexing result of shape [8, 6]